# Running cellpose 2.0 in colab on PC
# This notebook is to use a segmentation model 


In [ ]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob



# Use custom model to segment images

Take custom trained model, or upload your own model to google drive / colab runtime.

## Parameters

In [1]:
# model name and path

#@markdown ###Custom model path (full path):

model_path = input("Enter the path of the model file in your computer:")   #@param {type:"string"}
model_path= model_path.replace('\\', '/')

#@markdown ###Path to images:

dir = input("Enter the path of the folder that contains the image you want to segment in your computer:")  #@param {type:"string"}
dir= dir.replace('\\', '/')
#@markdown ###Channel Parameters:

Channel_to_use_for_segmentation = "Grayscale" #@param ["Grayscale", "Blue", "Green", "Red"]

# @markdown If you have a secondary channel that can be used, for instance nuclei, choose it here:

Second_segmentation_channel= "None" #@param ["None", "Blue", "Green", "Red"]


# Here we match the channel to number
if Channel_to_use_for_segmentation == "Grayscale":
  chan = 0
elif Channel_to_use_for_segmentation == "Blue":
  chan = 3
elif Channel_to_use_for_segmentation == "Green":
  chan = 2
elif Channel_to_use_for_segmentation == "Red":
  chan = 1


if Second_segmentation_channel == "Blue":
  chan2 = 3
elif Second_segmentation_channel == "Green":
  chan2 = 2
elif Second_segmentation_channel == "Red":
  chan2 = 1
elif Second_segmentation_channel == "None":
  chan2 = 0

#@markdown ### Segmentation parameters:

#@markdown diameter of cells (set to zero to use diameter from training set):
diameter =  9 #@param {type:"number"}
#@markdown threshold on flow error to accept a mask (set higher to get more cells, e.g. in range from (0.1, 3.0), OR set to 0.0 to turn off so no cells discarded):
flow_threshold = 1 #@param {type:"slider", min:0.0, max:3.0, step:0.1}
#@markdown threshold on cellprob output to seed cell masks (set lower to include more pixels or higher to include fewer, e.g. in range from (-6, 6)):
cellprob_threshold=-6 #@param {type:"slider", min:-6, max:6, step:1}


## run custom model

how to run the custom model in a notebook

In [47]:
# gets image files in dir (ignoring image files ending in _masks)
files = io.get_image_files(dir, '_masks')
print(files)
images = [io.imread(f) for f in files]

# declare model
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

# use model diameter if user diameter is 0
diameter = model.diam_labels if diameter==0 else diameter

# run model on test images
masks, flows, styles = model.eval(images, 
                                  channels=[chan, chan2],
                                  diameter=diameter,
                                  flow_threshold=flow_threshold,
                                  cellprob_threshold=cellprob_threshold
                                  )

['E:/Microscopy/Rawdates/20230510_DnaQ-Halo_Mutants4/Images/seg/wt\\_DnaQ_Halo_wt_20m_100ms_int_18per_561_03_seg.tif', 'E:/Microscopy/Rawdates/20230510_DnaQ-Halo_Mutants4/Images/seg/wt\\_DnaQ_Halo_wt_20m_100ms_int_18per_561_06_seg.tif', 'E:/Microscopy/Rawdates/20230510_DnaQ-Halo_Mutants4/Images/seg/wt\\_DnaQ_Halo_wt_20m_100ms_int_18per_561_09_seg.tif', 'E:/Microscopy/Rawdates/20230510_DnaQ-Halo_Mutants4/Images/seg/wt\\_DnaQ_Halo_wt_300ms_int_3per_561_03_seg.tif', 'E:/Microscopy/Rawdates/20230510_DnaQ-Halo_Mutants4/Images/seg/wt\\_DnaQ_Halo_wt_300ms_int_3per_561_06_seg.tif', 'E:/Microscopy/Rawdates/20230510_DnaQ-Halo_Mutants4/Images/seg/wt\\_DnaQ_Halo_wt_300ms_int_3per_561_09_seg.tif', 'E:/Microscopy/Rawdates/20230510_DnaQ-Halo_Mutants4/Images/seg/wt\\_DnaQ_Halo_wt_300ms_int_3per_561_12_seg.tif', 'E:/Microscopy/Rawdates/20230510_DnaQ-Halo_Mutants4/Images/seg/wt\\_DnaQ_Halo_wt_300ms_int_3per_561_15_seg.tif']


## save output to *_seg.npy

you will see the files save in the Files tab and you can download them from there

In [48]:
from cellpose import io

io.masks_flows_to_seg(images, 
                      masks, 
                      flows, 
                      diameter*np.ones(len(masks)), 
                      files, 
                      [chan, chan2])

## save output masks to tiffs/pngs or txt files for imageJ

In [49]:
io.save_masks(images, 
              masks, 
              flows, 
              files, 
              channels=[chan, chan2],
              png=True, # save masks as PNGs and save example image
              tif=True, # save masks as TIFFs
              save_txt=True, # save txt outlines for ImageJ
              save_flows=False, # save flows as TIFFs
              save_outlines=False, # save outlines as TIFFs 
              )
    